In [ ]:
import requests
import json
import time
from tqdm import tqdm
# from multiprocessing import Pool
# from multiprocessing import Process, Queue
# import multiprocessing as mp
import threading
from math import *


def jdump(obj): return json.dumps(obj, sort_keys=True, indent=4)


def jget(url, i):
    # print('jget attmpt ' + str(i))
    response = ''
    try:
        response = requests.get(url)
    except Exception as e:
        print('ERR: jget requests.get failed.')
        print('If you are using an internet connection provided by a school or organization with heavy internet traffic control, that may be why. :)')
        print('Exception: ' + str(e))
        print('Attmpt ' + str(i))
        print('url: ' + url)
        quit()
    else:
        if response.status_code == 200:
            # print('response 200, returning data')
            return json.loads(jdump(response.json()))
        elif i < 10:
            print(str(response.status_code) +
                  '; Failed to fetch data, trying again in one second.')
            time.sleep(1)
            jget(url, i+1)
        else:
            sc = response.status_code
            print('Attempted to get data 10 times, failed.')
            print('Error code: ' + str(sc) + '.')
            if sc == 301:
                print('The server is redirecting you to a different endpoint.')
            elif sc == 400:
                print('The server thinks you made a bad request. Incorrect credentials.')
            elif sc == 401:
                print('The server thinks you’re not authenticated.')
            elif sc == 403:
                print(
                    'The resource you’re trying to access is forbidden. You do not have the right permissions.')
            elif sc == 404:
                print('The resource you tried to access wasn’t found on the server.')
            elif sc == 503:
                print('The server is not ready to handle the request.')
            else:
                print('Unknown error type.')
            print('URL: ' + url)
            quit()


f = open('../json/users.json')
users_info = json.load(f)
usernames = users_info['usernames']
follows = users_info['follows']
f.close()


In [ ]:
c = len(follows)

userData = []
sampleData = []
threads = []


def col(K):
    # print('\t\tProcess '+str(K))
    batch = 0
    username = usernames[K]
    valid = jget('https://api.scratch.mit.edu/accounts/checkusername/' +
                 username, 0)['msg'] == 'username exists'
    if valid:
        user = jget('https://api.scratch.mit.edu/users/' +
                    username, 0)  # Data from each user
        projects = 40
        off = 0
        while projects > 39:
            i = batch | K << 10
            # print(username)
            userData.append((i, user))  # Add the user data
            sample = jget('https://api.scratch.mit.edu/users/' +
                          username+'/projects?limit=40&offset=' + str(off), 0)  # Get batch of projects
            # return sample
            # print('\t\tAppending '+username)
            sampleData.append((i, sample))  # Sample of 20 projects
            # print(len(sample))
            projects = len(sample)
            off += 40
            batch += 1


print("Collecting")
for K in tqdm(range(c)):
    t = threading.Thread(target=col, args=(K,))
    t.start()
    threads.append(t)

print("Consolidating")
for t in tqdm(threads):
    t.join()

print('Data collected.')


In [ ]:
print('sorting')
userData = sorted(userData, key=lambda D: D[0])
sampleData = sorted(sampleData, key=lambda D: D[0])

for K in range(len(userData)):
    userData[K] = userData[K][1]
for K in range(len(sampleData)):
    sampleData[K] = sampleData[K][1]

print('sorted')


In [ ]:
data = 'User,,,,Average Project Statistics,,,,,,,\nUsername,Followers,Country,Join Date,Views,Loves,Favorites,Remixes,Public,Published,Visible,Commentable,Projects\n'

print('--  Parsing data  --')

user = []
pusr = []
sample = []
views = 0
loves = 0
favorites = 0
remixes = 0
public = 0
published = 0
visible = 0
commentable = 0
count = 0
projects = ''
username = userData[0]['username']
country = ''
joinDate = ''

samples = len(userData)

for K in tqdm(range(samples+1)):
    if K < samples:
        user = userData[K]  # Each user
        pusr = username  # Previous username
        username = user['username']
        sample = sampleData[K]  # Each sample of projects
    else:
        username = pusr+'a'

    if not pusr == username:
        # Stringify
        views = str(views)
        loves = str(loves)
        favorites = str(favorites)
        remixes = str(remixes)
        public = str(public)
        published = str(published)
        visible = str(visible)
        commentable = str(commentable)

        # Append previous user, as username changed
        # ldata.append(pusr, int(follows[K].strip()), country, joinDate, views, loves,
        #         favorites, remixes, public, published, visible, commentable, projects)

        data += pusr+','+follows[K].strip()+','+country+','+joinDate+','+views+','+loves + ',' + \
            favorites+','+remixes+','+public+',' + published + \
            ','+visible+','+commentable+','+projects+'\n'

        # Reset parameters
        views = 0
        loves = 0
        favorites = 0
        remixes = 0
        public = 0
        published = 0
        visible = 0
        commentable = 0
        count = 0

        if K == samples:
            break

    # User data
    country = user['profile']['country']
    joinDate = user['history']['joined'].split('T')[0]

    for i in range(len(sample)):  # Total data in single sample
        project = sample[i]
        views += project['stats']['views']
        loves += project['stats']['loves']
        favorites += project['stats']['favorites']
        remixes += project['stats']['remixes']
        public += 1 if project['public']else 0
        published += 1 if project['is_published']else 0
        visible += 1 if project['visibility'] == 'visible'else 0
        commentable += 1 if project['comments_allowed']else 0

        if project['public'] and project['is_published'] and project['visibility'] == 'visible':
            count += 1  # Valid public, published, and visible project

    projects = str(count)  # Project count

    # Data append at username change

f = open('dataset.csv', 'w')
f.write(data)
f.close()
print('File written to dataset.csv.')
